이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 12.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a051dc3356489e2db2daffc911fcf3ba48c0630c00bea8ea3f7487a62f6cf816
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


**Spark Session**을 하나 만든다

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark Schema Evolution Demo") \
        .master("local[3]") \
        .getOrCreate()

In [3]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/schema1.parquet
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/schema2.parquet
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/schema3.parquet

--2023-01-26 23:42:10--  https://s3-geospatial.s3.us-west-2.amazonaws.com/schema1.parquet
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.92.209.162, 52.92.176.250, 52.218.184.129, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.92.209.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49915 (49K) [binary/octet-stream]
Saving to: ‘schema1.parquet’

schema1.parquet     100%[===================>]  48.75K  --.-KB/s    in 0.08s   

2023-01-26 23:42:10 (607 KB/s) - ‘schema1.parquet’ saved [49915/49915]

--2023-01-26 23:42:11--  https://s3-geospatial.s3.us-west-2.amazonaws.com/schema2.parquet
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.92.209.162, 52.92.176.250, 52.218.184.129, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.92.209.162|:443... con

In [4]:
df1 = spark.read. \
    parquet("schema1.parquet")
    
df1.printSchema()
df1.show()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)

+-------------------+------------------+------------------+------------------+------------------+
|               Date|              Open|              High|               Low|             Close|
+-------------------+------------------+------------------+------------------+------------------+
|2010-01-04 08:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|
|2010-01-05 08:00:00|        214.599998|        215.589994|        213.249994|        214.379993|
|2010-01-06 08:00:00|        214.379993|            215.23|        210.750004|        210.969995|
|2010-01-07 08:00:00|            211.75|        212.000006|        209.050005|            210.58|
|2010-01-08 08:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|
|2010-01-11 08:00:00|212.79

In [5]:
df2 = spark.read. \
    parquet("schema2.parquet")
    
df2.printSchema()
df2.show()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)

+-------------------+------------------+------------------+------------------+------------------+---------+
|               Date|              Open|              High|               Low|             Close|   Volume|
+-------------------+------------------+------------------+------------------+------------------+---------+
|2010-01-04 08:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|
|2010-01-05 08:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|
|2010-01-06 08:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|
|2010-01-07 08:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|
|2010-01-08 08:0

In [6]:
df3 = spark.read. \
    parquet("schema3.parquet")
    
df3.printSchema()
df3.show()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 08:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 08:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 08:00:00|        214.379993|            215.23|        210.750004|        210.969995

In [7]:
df = spark.read. \
    option("mergeSchema", True). \
    parquet("*.parquet")
    
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [8]:
df.show(10)

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 08:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 08:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 08:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 08:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 08:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    